# ARP Lab
## A Retirement Planning Laboratory

This package is a retirement modeling framework for exploring the sensitivity of retirement financial decisions. Strictly speaking, it is not a planning tool, but more an environment for exploring *what if* scenarios. It provides different realizations of a financial strategy. One can certainly have a savings plan, but due to the volatility of financial investments, it is impossible to have a certain asset earnings plan. This does not mean one cannot make decisions. These decisions need to be guided with an understanding of the sensitivity of the parameters.This is exactly where this tool fits it. Given your savings and spending desires, it can generate different future realizations of your strategy under different market assumptions, helping to better understand your financial situation.

Copyright - Martin-D. Lacasse (2023)

Disclaimers: *I am not a financial planner. You make your own decisions. This program comes with no guarantee. Use at your own risk.*

# Exploring Bengen's 4% rule of thumb with ARP Lab

## Introduction
This file is provided as an example to introduce you to ARP Lab.

It is assumed that you have some familiarity with using a jupyter notebook or jupyterLab, and some basic programming skills in Python. If not, a simple tutorial can guide you to the basic skills needed.

For simulating your own realizations, use the files beginning with *template*. Make a copy and rename them keeping the same extension and give them you own names. Then you'll be able to personalize a case with your own numbers and start experimenting with ARP Lab.

This notebook describes the case of Sam, a fictitious individual used for demonstrating the conditions required for the so-called 4% rule. This case considers no pension, no social security benefits, just a portfolio of a tax-deferred account with 50% stocks and 50% bounds. This scenario is then tested to survive sequences of historical rates, particularly the ones which happened in 1966.

The article that popularized the 4% rule is from William Bengen (Journal of Financial Planning, October 1994). The scenario consists of looking at the longevity of a 50% common stocks and 50% intermediate-term Treasury notes portfolio under a variety of withdrawal rates and depending on the historical year that the individual started her retirement. Cases considered consisted of withdrawal rates from 1% to 6%, to be determined on the balance of a single tax-deferred account at the first year of retirement and then be indexed for inflation. These scenarios highlight the effect of sequence of returns through using historical data.

The original claim is that a withdrawal rate of 4% of the portfolio at the first year of retirement would be sustainable for a 30-year retirement period. These withdrawals are then adjusted for inflation and no income tax is considered. To avoid triggering the income tax machinery in ARP Lab, we will use an unrealistically small amount of \\$2k per year to be withdrawn from a single \\$50k tax-deferred account. As this scenario is meant to explore the effects of a percentage, multiplying all the numbers by 20 will convert the results of withdrawing \\$40k from a \\$1M portfolio. This can obviously be done for any other portfolio balance of interest.

Note that Bengen used data from Ibbotson's SBBI (Stocks, Bonds, Bills, Inflation) Yearbook while the data used in ARP Lab is from the NYU Stern business school. There will be slight discrepancies in the number of years that the accounts can sustain due to the slight difference in the rates used. As a results, the investment portfolios in ARP Lab last a little shorter than those originally obtained by Bengen. The trends and conclusions are the same however.

### Just some Python module bookkeeping
The following commands are bookkeeping calls that load the required module and set the program to display verbose informational messages as it runs. It also makes sure that all graphs are properly displayed within the jupyter interface.

These commands need to be at the beginning of any ARP Lab notebook.

In [ ]:
%matplotlib inline
# import arp

import importlib
arp = importlib.import_module('arp')
importlib.reload(arp)

arp.setVerbose(True)

## Initializing the realization
In order to be able to generate a realization of the future, one must start with providing the year of birth of each spouse(s) and their expected lifespan(s).

For selecting your own numbers, there are plenty of longevity predictors on the web. Pick your favorite:

https://longevityillustrator.org

https://www.livingto100.com/calculator

https://www.sunlife.ca/en/tools-and-resources/tools-and-calculators/life-expectancy-calculator/

or just Google life expectancy calculator.

There are two values needed for couples. Single individuals just enter one value in each list between square brackets `[ ]`. For couples, always keep the same order in the pair of values when entering the data.

When first creating a plan, the default values used for assets allocation ratios and rates of return will be reported. These values are listed to make you aware that these values would be used if no other choice is entered. We will cover how to do this in the next sections.

Here we are reproducing the 4% withdrawal rule. Sam is 63 years old and thinks being able to live until 93 years old.

In [ ]:
plan = arp.Plan(YOB=[1961], expectancy=[93])

## Specify account balances and spousal beneficiaries
For each spouse, savings accounts have three buckets comprising of the total value of:
- Individual **taxable** investment or savings accounts, including bank accounts, and CDs - do not include your safety net account which should typically be sufficient for sustaining 6 months of living expenses;
- **Tax-deferred** savings accounts, including all IRAs, 401k, 403b, etc.;
- **Tax-free** savings accounts, including Roth IRAs and Roth 401k.

For married couples, each spouse will have to enter values for each type of savings account, following the same order as before. For single individuals, only one value is needed between square brackets `[ ]`.

Most investment accounts have named beneficiaries. The *beneficiary* values specify the fraction of total of assets left to the other spouse at death. For example, a spouse leaving 3/4 of her fortune to her three children and the other part to her partner would have a beneficiary value of 0.25. While this number is irrelevant for single individuals, it still needs to be entered: just use `[1]`.

Sam has \\$50k in a single tax-deferred account for a %4 withdrawal. This amount is ridiculously small to avoid triggering income tax. Just multiply all numbers by 20 in your head to get to a \\$1M account. 

In [ ]:
plan.setAssetBalances(taxable=[0], 
                      taxDeferred=[50000],
                      taxFree=[0],
                      beneficiary=[1])

## What are the current and future assets allocations?
Each savings account can invest in 4 major classes of assets:
- Equity funds tracking the S&P 500 index;
- Bond assets tracking the Corporate bonds (Baa) index;
- Fixed-income securities represented by the performance of 10-year Treasury notes;
- Inflation-indexed securities tracking the urban Consumer Price Index (common assets).

The total of percentages in each class of assets for each savings account must add to 100%.

You are asked to provide assets allocation ratios for today, and ones for the time at the end of your life.
Values in between will be interpolated using a linear operator (for now). This can be useful
if you want to shift assets allocation as you age.


For our particular 4% rule case, Sam's assets allocations use a 50/50 portfolio per Bengen's study. Note that assets allocation ratios are entered as percentages, and that these percentages are for each type of savings account.

In [ ]:
plan.setInitialAR(taxable=[[50, 0, 50, 0]],
                  taxDeferred=[[50, 0, 50, 0]],
                  taxFree=[[50, 0, 50, 0]])

plan.setFinalAR(taxable=[[50, 0, 50, 0]],
                taxDeferred=[[50, 0, 50, 0]],
                taxFree=[[50, 0, 50, 0]])

plan.interpolateAR('linear')

## What about anticipated fixed income?
Pension and social security are fixed income. Model here assumes that pension income is not inflation adjusted while social security benefits are (but ARP Lab can easily be modified to account for inflation-adjusted pensions). Numbers to be provided are the predicted amount for each spouse and the age of the commencement of benefits.

By default, no pension benefits are assumed. This can also be specified explicitly by entering zeros (0) as entries, as in

    plan.setPension([0, 0], [65, 65])
    
For social security, one must provide the predicted amount(s) and the starting age(s) at which benefits are anticipated to be received. There are plenty of social security benefit estimators on the web, including the info you can get directly from your own account at the Social Security Administration (ssa.gov). Another interesting calculator can be found at www.opensocialsecurity.com. This calculator allows you to compare different scenarios regarding your commencement age through a sensitivity plot.


For modeling the 4% rule, we assume that Sam has no pension nor social security benefits.

In [ ]:
plan.setPension([0], [65])
plan.setSocialSecurity([0], [65])

## There must be a plan for savings
The most manageable part of retirement planning is the control one has over work income, contributions to savings accounts, Roth conversions, and other big spending items in the near- and mid-term future.
In order to execute a realization, one must provide an earning, saving, and Roth conversion plan. This is done through providing an Excel workbook with one spreadsheet (tab) per spouse with the following information:

|year|anticipated income|ctrb taxable | ctrb 401k | ctrb Roth 401k | ctrb IRA | ctrb Roth IRA | Roth X | big ticket items|
|--|--|--|--|--|--|--|--|--|
|2024 | | | | | | | | |
|2025 | | | | | | | | |
| ... | | | | | | | | |
|20XX | | | | | | | | |

Here, 20XX is the last row that is the first year after the last spouse has passed based on the life expectancy values provided. For the columns, *anticipated income* is the annual amount (gross) that you anticipate to receive from employment or other sources (not including dividends from your taxable investment account). Note that column names are case sensitive and all of these entries must be in lower case. Best way to start this process is to use the template provided rightly named *template.xlsx*.

For the purpose of this exercise, there is no clear definition of retirement age. There will be a year, however, from which you will stop having anticipated income, or diminished income due to decreasing your work load. This transition can be gradual or sudden. Therefore there is no need to enter a retirement age for the sole purpose of quantifying your financial future.

Contributions to your savings accounts are marked as *ctrb*. Contributions to your 401k must also include your employer's contributions. As this file is in Excel, one can use the native calculator to enter a percentage of the anticipated income for contributions as this can sometimes be easier.

Roth conversion are specified in the column marked *Roth X*. Roth conversion are typically performed in the years when the income is lower (and therefore lower tax rates), typically in the bridge years between having a full-time regular salary and collecting social security.

Finally, *big ticket items* are used for accounting for the sale or purchase of a house, or any other major expense or money that you would give or receive (e.g., inheritance, or large gifts to or from you). Therefore, the sign (+/-) of entries in this column is important. All other column entries should be positive.

The tab name for each spreadsheet represents the name of the spouse for reporting yearly transactions affecting the plan. In fact, ARP Lab extracts the names of these tabs to determine the individuals' names. Therefore, you need to rename these tabs to reflect your personal names if you want to accurately represent your case.

Note that the file format from the (free) LibreOffice software can also be read, so you do not need to have an Excel license.

Sam has no additional income and does not make any additional contributions. We will use the template file which is empty but has the first tab named *Sam*.

In [ ]:
plan.readContributions('template.xlsx')

## How much net income is desirable at retirement?
For determining the desirable annual net income in retirement, certified planners will strongly suggest that you've must have already done a cash flow analysis on your yearly spending. After this exercise, you should have a good idea of how much you'll need in retirement. Another approach is to experiment with multiple spending scenarios and see what your current and future savings can sustain under different market conditions. As one nears retirement, both approaches will need to meet at one point.

The desired income defined here is the minimum annual **net** income (i.e., after paying federal income tax) that one would like to have starting at her/his "retirement age" (we will provide a loose definition of the term *retirement age* below). This desired income must be adjusted for inflation and can follow an additional adjustment called a *smile* profile. A *smile* profile accounts for the fact that your spending capacity will modulate during retirement as you go from the so-called gogo years to the no-go years. A *flat* profile, on the other hand, will keep the same value, which will only be adjusted for inflation.

The target and actual net income values achieved by the realization can be plotted as will see below.


Sam is applying the 4% rule. As the assets are \\$50k, $2000 is the first withdrawal. To get results for higher savings balances, say \\$1M, just multiply all numbers by 20.

In [ ]:
plan.setDesiredIncome(2000, 'flat')

## How to withdraw from spousal accounts?
It can be desirable for couples to determine how to make distributions from each spousal account.
By specifying an *auto* spousal split, the withdrawals from the retirement savings accounts will be made proportional to their respective balances. It can also be specified as a fixed fractional value (e.g., 0.65) in which case 0.65 will be taken from the first account and the other 35% taken from the other (second entry) spouse, at this risk of depleting one account before the other, and running short on income.

All withdrawals use a smart banking approach which favors depleting taxable accounts before tax-deferred, before tax-free. With *auto*, additional checks and bounds are used to better coordinate spousal accounts.


A spousal split is irrelevant to Sam who is filing as a single individual. Just set it to 1.

In [ ]:
plan.setSpousalSplit(1.)

## Specify rates of return and inflation rate
Rates of return for each class of assets can be specified, including the rate of inflation.
These are important components for  future assumptions.
Setting future rates is done with the `setRates()` method from which one can select different
sources for determining the rates.
Valid choices are *historical*, *stochastic*, *average*, 'fixed', or 'default'.

For the *historical*, *stochastic*, and *average* options, data from 1928 to the last year are available for experimenting.
Ranges chosen smaller than the life horizon of the longest-lived individual will have rate values repeated in cycle. For example,
choosing historical data from 1994 to (up and including) 1996 will repeat these three values over the time span of the realization.
This would be called as follows:

    plan.setRates('historical', 1994, 1996)

This case is only provided as an explanatory example, as it would have little practical value.

If the upper bound is not provided as the third argument, then the latest data year (i.e., last year) will be assumed by default.
If one chooses a historical range starting from 1970, ARP Lab will use the rates of 1970 for this year and 1971 for next, etc. This would be simulated as follows:

    plan.setRates('historical', 1970)
  
Due to its particular sequence of rates, the worst-case historical scenario is a retirement starting in 1966. This can be simulated as follows:

    plan.setRates('historical', 1966)

In this case, the  current year will have the same rates as those that happened in 1966, and next year will have those from 1967, and so on. This choice is given for instructional purposes only. No one should make a plan based on the worst-case historical scenario. Nevertheless, it can be informative to test your own case. In practice, however, a success rate larger than 90% over a reasonable set of historical starting years and market assumptions would be acceptable by a large portion of rational thinkers. But this is all a personal choice. We'll cover how to model multiple starting years below.

Alternatively, one can choose a *stochastic* approach in which case the rates are determined from the multivariate distribution for the 4 rates in the selected year range.  The computed statistical distribution of the selected range of data is used to generate random rate values. For example,

    plan.setRates('stochastic', 1945)
    
will analyze the annual rates from 1945 up to last-year and compute means and covariance to generate new data that are statistically representative of the ones observed during this selected time period. The rates randomly generated for the time span can be plotted and examined as we will see below. Similarly,

    plan.setRates('stochastic', 1940, 1970)

would generate random rates consistent to those observed during the 1940 - 1970 time period.

Rates can also be set to fixed values obtained from an average over a time interval using the *average* option. For example, the call

    plan.setRates('average', 1990, 2020)

would set the rates to fixed values being the average observed from 1990 to 2020 inclusively.
    
Finally one can also use fixed annual rates by providing a list of 4 entries in percent as follows:

    myrates = [9.6, 4.0, 3.0, 3.8]
    plan.setRates('fixed', values=myrates)
    
This example would use fixed rates of 9.6%, 4.0%, 3.0%, and 3.8% as average annual returns on S&P 500, corporate bonds, Treasury notes, and common assets, respectively, with an average annual inflation rate of 3.8% for the full duration of the time simulation. Recall that the common assets class consists of investments tracking inflation only. Therefore the last index serves both to track the common asset class and to adjust values for inflation.

Also note that the S&P 500 rates provided always include dividends, which are assumed to be reinvested.


We are interested to know if Sam's situation would survive a retirement started in 1966. The following call would model exactly that situation.

In [ ]:
plan.setRates('historical', 1966, 1997)

Since ARP Lab has the historical rates available, one can also display their histograms using a simple function call. This is done with `arp.showRateDistribution()`.

Given the standard deviation of each histogram, the risk/benefit between stocks and bonds is clear. As this function returns the means and covariance matrix, we assign the result to the `_` variable which, in Python, is used to store results from the last command.

In [ ]:
_ = arp.showRateDistributions(1966, 1996)

## Generating the outcome of a scenario
We're now ready to run a single instance of a scenario. 

Recall that we set the rates above to mimic those from and after 1966. For this purpose, we only need to run a single realization looking at what would happen if Sam had retired in 1966. So let's run this plan: The following call runs all the required calculations for the time horizon over Jack and Jill's life expectancies. `run()` returns many variables that we assign to the `_` variable, used to store the answer to the last command in Python. Just a trick to avoid printing all these numbers on screen. Having set the variable to verbose, `run()` will report what happens during each year modeled.

In [ ]:
_ = plan.run()

# Analysis

### Show net income compared to target income over the years
This graph shows how the actual net income generated by the plan realization matches the inflation-adjusted net income profile specified. We can see that Sam runs out of money in the last year.

In [ ]:
plan.showNetIncome('Sam 1966');

### Show sources of income over the years
Income will typically come from multiple sources, and it can be quite complex. This graphs shows the breakdown of Jack and Jill's sources of income by spouse and by origin. Note that distributions from tax-deferred accounts (*dist*) are distinguished from required minimum distributions (*rmd*) as they serve different purposes. Other labels should be self-explanatory. An extra title label *Sam* is passed to the function.

In [ ]:
plan.showSources('Sam 1966');

### Show savings accounts balances at the beginning of each year
The balance for each savings account for each spouse is calculated at the beginning of each year. Another important aspect of this graph is how much is left at the end of the realization. This will be addressed in the next cell.

In [ ]:
plan.showAccounts('Sam 1966');

To get more information about accounts at the last year of the realization, one can use

    plan.setDeferredTaxRate(30)
    plan.estate()
    
which returns the value of the assets in today's \\$ at the last year of the scenario, assuming (in this case) a 30\% tax burden on the taxable portion of the estate (read tax-deferred savings accounts). The `estate()` function returns two values: the total post-tax value of all savings account in today's dollars and the cumulative inflation rate between today and the last day of the realization. We can also override the tax-deferred rate value by giving an argument. In the case below, a value of 35% would override the value shown above.

In [ ]:
plan.estate(0)

### Show assets allocations during the time period
The allocation of assets is shown for the three types of savings accounts (taxable, tax-deferred, and tax-free) for each spouse and for the 4 types of investments: stocks (S&P 500), corporate bonds (Baa), Treasury notes (10-y), and common assets tracking inflation.

In [ ]:
plan.showAssetDistribution('Sam 1966')

### Show annual taxes paid over the years of the realization
This graph shows how much Jack and Jill paid in federal taxes and IRMAA income-related Medicare insurance monthly adjustments over the years of this realization.

In [ ]:
plan.showTaxes('Sam 1966');

### Show taxable annual income and anticipated tax brackets
Gross income also includes Roth conversions and big-ticket items, both of which are not contributing to your net income. This graph shows Jack and Jill's gross taxable income and how it compare with some anticipated Federal tax marginal brackets. This visualization is very convenient when one wants to perform Roth conversions and remain below a certain tax bracket.

Note the shift in tax brackets taking place as the Tax Cut and Job Act expires after 2025.

In [ ]:
plan.showGrossIncome('Sam 1966');

### Show annual rates used for calculations
As described above, there are many choices for selecting rates. This graph will display the annual rates used during the time span of this realization.

In [ ]:
plan.showRates('Sam 1966');

## Can we save this realization?
This instance of a future realization contains information on the distribution amounts, including the required minimum distribution that had to be performed under the given assumptions. This info can be saved in an excel workbook with one spreadsheet (tab) for each spouse. Worksheet will also contain annual rates, income, income taxes, and account balances. Second `True/False` argument controls if existing files get overwritten. Also remember that Windows will not allow the file to be overwritten while the file is being opened in Excel. In that case, the script will ask you to close the file and will retry to save.

Here, this call will create an excel workbook with one spreadsheet (tab) for Jack and one for Jill. 

Open the file in Excel to see what it looks like. 

In [ ]:
plan.saveInstance('Sam 1966', True)

## Next level
ARP Lab allows one to run multiple simulations in one call. For example, one can run multiple historical cases over a range of years. In those simulations, multiple instances are executed to further explore the robustness of one's decisions under different market realizations. In another case, one can run a Monte-Carlo simulation in which fictitious rates derived from the statistical distribution of those observed in a selected time period are used for the time span of the simulation.


In these simulations, multiple scenarios will be generated and need to be compared. For ranking outcomes, it is instructive to compare the final value of the estate, if any. Given that parts of the assets portfolio are taxable, one has to provide an estimate of the income tax rate to be applied to the taxable portion of the final assets. This is done using the `setDeferredTaxRate()` method, by which an estimated tax rate is applied on the tax-deferred portion of the estate for comparison in the last year of the scenario. The default value is 25%. Note that this rate is the marginal tax rate that will be paid by the heirs, not including possible additional estate taxes.

In [ ]:
plan.setDeferredTaxRate(0)

### Historical comparisons
The `runHistorical()` function runs multiple scenarios each going through a historical sequence of rates starting at a given year. It simulates a situation in which this year's rates would be those which happened. This scenario simulates future rates based on a past sequence. As such, the next simulation year would take the rates which happened in 1961 and so on.

User can select what values to plot at the end of each scenario through the `myplots` keyword. It understands the following words: *net income*, *rates*, *taxable income*, *sources*, *taxes*, *accounts*, and *allocations*.

We will run Sam's scenario over many years. Notice the difference in money left at the end of each sequence. Let's increase the desired income by %.05 from %4.00 to %4.05 of the initial savings balance.

In [ ]:
plan.setDesiredIncome(2000, 'flat')
arp.setVerbose(False)
plan.runHistorical(1960, 1988, myplots=['rates', 'net income'], tag='Sam')

As we can see, all scenarios but one are successful (i.e., the case of the 1966 rates which fails in the 29th year) with a 4% initial withdrawal rate. But real-life situations are often more complex when one considers social security benefits and other income. When income tax and Roth conversions are considered, the resulting situation is even more challenging.

### Monte-Carlo simulations
In a Monte-Carlo simulation, a random number generator is used to create values that meet the statistical criterion of a known distribution. In ARP Lab, one specifies the range of years of historical annual rates data that will be used to characterize the probability distribution of a normal multivariate distribution. Once selected, one can run as many realizations as desired. The more is generally the better, as it will improve the accuracy of the mean value while increasing the possibility of observing rare events. A value of 1,000 is generally more than sufficient. For the sake of demonstration in this notebook, we will use a smaller number of iterations of 500.

In [ ]:
plan.runMonteCarlo(500, 1960, 2022)

## One more notch up for modeling
We can also run Roth conversion estate maximization strategies. Using commutativity of multiplication, one can easily demonstrate that there is no point in doing a Roth conversion if the tax rate to be paid on the conversion now is the same as the one to be paid on the distribution later. However, things are much more complex in reality, and one has to take RMDs in consideration on top of all other unknown factors that can happen, including changes in the tax code. For this purpose, a simple heuristical Roth optimizer maximizes the estate amount left at the passing of the last spouse, considering a tax bracket on the taxable portion of the estate, and under the constraint of receiving a desired income. Function will return a new plan, and an array containing proposed Roth conversions.

Here is an example of how it works: 

In [ ]:
# Use time reproducible rates
plan.setRates('average', 1928, 2022)
plan.setDesiredIncome(2000, 'flat')
plan2, conversions = arp.optimizeRoth(plan, 0)
print(conversions)
plan.run();
plan2.run();
plan2.saveInstance('plan2', True)

The Excel file named *instance_plan2.xlsx* saved will contain all the information for the heuristically optimized plan. Take a few minutes to open this file and explore the different tabs.

In this case, Roth conversions should not make any difference as all the income is below the lowest tax bracket on purpose.

In [ ]:
plan.showNetIncome('original')
plan2.showNetIncome('conversion')
plan.showSources('original')
plan2.showSources('conversion')
plan.showAccounts('original')
plan2.showAccounts('conversion')

### Saving configuration parameters
Configuration parameters set for a run can be saved in a file, thus avoiding to re-enter the same information in the future. This is done using the `savePlan()` and `readPlan()` function calls. Note that 'readPlan()' returns a newly created plan pre-configured according to the parameters set in the configuration file.

In [ ]:
# To save all configuration parameters
arp.savePlan(plan, 'sam')
             
# To create a new plan configured according to the parameters in configuration file.
newplan = arp.readPlan('sam')

This notebook is provided as a basic example on what you can do to assess the robustness and sensitivity of your retirement strategy. Go and explore!


Enjoy!
